In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import statsmodels.api as sm

In [ ]:
# Load the dataset

mortality_data = pd.read_excel('/content/drive/MyDrive/Data/stmf.xlsx')
covid_cases_data = pd.read_csv('/content/drive/MyDrive/Data/owid-covid-cases.csv')

In [ ]:
mortality_data

,CountryCode,Year,Week,Sex,D0_14,D15_64,D65_74,D75_84,D85p,DTotal,R0_14,R15_64,R65_74,R75_84,R85p,RTotal,Split,SplitSex,Forecast
0,AUS,2015,1,m,17.166833,358.833167,250.0,436.0,413.0,1475.0,0.000386,0.002373,0.013133,0.045643,0.127992,0.006484,1,0,0
1,AUS,2015,1,f,13.968728,199.031272,183.0,350.0,704.0,1450.0,0.000332,0.001309,0.009350,0.031185,0.125849,0.006289,1,0,0
2,AUS,2015,1,b,31.135561,557.864439,433.0,786.0,1117.0,2925.0,0.000360,0.001840,0.011215,0.037832,0.126633,0.006386,1,0,0
3,AUS,2015,2,m,17.473384,334.526616,255.0,386.0,422.0,1415.0,0.000393,0.002212,0.013395,0.040409,0.130781,0.006220,1,0,0
4,AUS,2015,2,f,11.490405,186.509595,162.0,322.0,675.0,1357.0,0.000273,0.001227,0.008277,0.028690,0.120665,0.005886,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118357,USA,2023,15,f,206.000000,5171.000000,5056.0,7294.0,10194.0,27921.0,0.000372,0.002521,0.014367,0.037380,0.138485,0.008656,0,0,1
118358,USA,2023,15,b,485.000000,14003.000000,11889.0,14890.0,16815.0,58082.0,0.000428,0.003399,0.017883,0.042412,0.144099,0.009095,0,0,1
118359,USA,2023,16,m,286.000000,8306.000000,6500.0,7642.0,6415.0,29149.0,0.000493,0.004015,0.020775,0.049003,0.148909,0.009224,0,0,1
118360,USA,2023,16,f,230.000000,4942.000000,4869.0,7135.0,10020.0,27196.0,0.000415,0.002409,0.013835,0.036565,0.136121,0.008431,0,0,1


In [ ]:
covid_cases_data

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316573,ZWE,Africa,Zimbabwe,2023-06-03,265139.0,32.0,21.000,5695.0,0.0,0.429,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316574,ZWE,Africa,Zimbabwe,2023-06-04,265139.0,0.0,16.571,5695.0,0.0,0.286,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316575,ZWE,Africa,Zimbabwe,2023-06-05,265139.0,0.0,15.571,5695.0,0.0,0.000,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
316576,ZWE,Africa,Zimbabwe,2023-06-06,265139.0,0.0,15.571,5695.0,0.0,0.000,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN


In [ ]:
covid_cases_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316578 entries, 0 to 316577
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    316578 non-null  object 
 1   continent                                   301554 non-null  object 
 2   location                                    316578 non-null  object 
 3   date                                        316578 non-null  object 
 4   total_cases                                 280456 non-null  float64
 5   new_cases                                   307848 non-null  float64
 6   new_cases_smoothed                          306584 non-null  float64
 7   total_deaths                                259613 non-null  float64
 8   new_deaths                                  307908 non-null  float64
 9   new_deaths_smoothed                         306678 non-null  float64
 

In [ ]:
# Filter the Dataset

mortality_df = mortality_data.drop(columns=['D0_14', 'D15_64', 'D65_74',
       'D75_84', 'D85p', 'R0_14', 'R15_64', 'R65_74', 'R75_84',
       'R85p', 'Split', 'SplitSex', 'Forecast'], axis=1)
covid_cases_df = covid_cases_data[['iso_code', 'date', 'total_cases', 'new_cases', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated']]

In [ ]:
mortality_df

,CountryCode,Year,Week,Sex,DTotal,RTotal
0,AUS,2015,1,m,1475.0,0.006484
1,AUS,2015,1,f,1450.0,0.006289
2,AUS,2015,1,b,2925.0,0.006386
3,AUS,2015,2,m,1415.0,0.006220
4,AUS,2015,2,f,1357.0,0.005886
...,...,...,...,...,...,...
118357,USA,2023,15,f,27921.0,0.008656
118358,USA,2023,15,b,58082.0,0.009095
118359,USA,2023,16,m,29149.0,0.009224
118360,USA,2023,16,f,27196.0,0.008431


In [ ]:
covid_cases_df

,iso_code,date,total_cases,new_cases,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,AFG,2020-01-03,NaN,0.0,NaN,NaN,NaN
1,AFG,2020-01-04,NaN,0.0,NaN,NaN,NaN
2,AFG,2020-01-05,NaN,0.0,NaN,NaN,NaN
3,AFG,2020-01-06,NaN,0.0,NaN,NaN,NaN
4,AFG,2020-01-07,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
316573,ZWE,2023-06-03,265139.0,32.0,NaN,NaN,NaN
316574,ZWE,2023-06-04,265139.0,0.0,NaN,NaN,NaN
316575,ZWE,2023-06-05,265139.0,0.0,NaN,NaN,NaN
316576,ZWE,2023-06-06,265139.0,0.0,NaN,NaN,NaN


In [ ]:
# Extract Week number and Year

covid_cases_df['date'] = pd.to_datetime(covid_cases_df['date'])
covid_cases_df

# covid_cases_df.loc[: , 'week'] = covid_cases_df['date'].dt.isocalendar().week
# covid_cases_df.loc[: ,'Year'] = covid_cases_df['date'].dt.year

<ipython-input-45-daf77b1ba35a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_cases_df['date'] = pd.to_datetime(covid_cases_df['date'])


,iso_code,date,total_cases,new_cases,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,AFG,2020-01-03,NaN,0.0,NaN,NaN,NaN
1,AFG,2020-01-04,NaN,0.0,NaN,NaN,NaN
2,AFG,2020-01-05,NaN,0.0,NaN,NaN,NaN
3,AFG,2020-01-06,NaN,0.0,NaN,NaN,NaN
4,AFG,2020-01-07,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
316573,ZWE,2023-06-03,265139.0,32.0,NaN,NaN,NaN
316574,ZWE,2023-06-04,265139.0,0.0,NaN,NaN,NaN
316575,ZWE,2023-06-05,265139.0,0.0,NaN,NaN,NaN
316576,ZWE,2023-06-06,265139.0,0.0,NaN,NaN,NaN


In [ ]:
covid_cases_df.loc[: , 'Week'] = covid_cases_df['date'].dt.isocalendar().week
covid_cases_df

<ipython-input-46-96ffbb2ea6c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_cases_df.loc[: , 'Week'] = covid_cases_df['date'].dt.isocalendar().week


,iso_code,date,total_cases,new_cases,total_vaccinations,people_vaccinated,people_fully_vaccinated,Week
0,AFG,2020-01-03,NaN,0.0,NaN,NaN,NaN,1
1,AFG,2020-01-04,NaN,0.0,NaN,NaN,NaN,1
2,AFG,2020-01-05,NaN,0.0,NaN,NaN,NaN,1
3,AFG,2020-01-06,NaN,0.0,NaN,NaN,NaN,2
4,AFG,2020-01-07,NaN,0.0,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...
316573,ZWE,2023-06-03,265139.0,32.0,NaN,NaN,NaN,22
316574,ZWE,2023-06-04,265139.0,0.0,NaN,NaN,NaN,22
316575,ZWE,2023-06-05,265139.0,0.0,NaN,NaN,NaN,23
316576,ZWE,2023-06-06,265139.0,0.0,NaN,NaN,NaN,23


In [ ]:
covid_cases_df.loc[: ,'Year'] = covid_cases_df['date'].dt.year
covid_cases_df

<ipython-input-47-ff5506d2f130>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_cases_df.loc[: ,'Year'] = covid_cases_df['date'].dt.year


,iso_code,date,total_cases,new_cases,total_vaccinations,people_vaccinated,people_fully_vaccinated,Week,Year
0,AFG,2020-01-03,NaN,0.0,NaN,NaN,NaN,1,2020
1,AFG,2020-01-04,NaN,0.0,NaN,NaN,NaN,1,2020
2,AFG,2020-01-05,NaN,0.0,NaN,NaN,NaN,1,2020
3,AFG,2020-01-06,NaN,0.0,NaN,NaN,NaN,2,2020
4,AFG,2020-01-07,NaN,0.0,NaN,NaN,NaN,2,2020
...,...,...,...,...,...,...,...,...,...
316573,ZWE,2023-06-03,265139.0,32.0,NaN,NaN,NaN,22,2023
316574,ZWE,2023-06-04,265139.0,0.0,NaN,NaN,NaN,22,2023
316575,ZWE,2023-06-05,265139.0,0.0,NaN,NaN,NaN,23,2023
316576,ZWE,2023-06-06,265139.0,0.0,NaN,NaN,NaN,23,2023


In [ ]:
# Fillna with Zero
covid_cases_df = covid_cases_df.fillna(0)

In [ ]:
print(covid_cases_df[covid_cases_df['Week'] == 52])
print(covid_cases_df[covid_cases_df['Year'] == 2023])

       iso_code       date  total_cases  new_cases  total_vaccinations  \
353         AFG 2020-12-21      50678.0      142.0                 0.0   
354         AFG 2020-12-22      50888.0      210.0                 0.0   
355         AFG 2020-12-23      51070.0      182.0                 0.0   
356         AFG 2020-12-24      51357.0      287.0                 0.0   
357         AFG 2020-12-25      51595.0      238.0                 0.0   
...         ...        ...          ...        ...                 ...   
316416      ZWE 2022-12-28     259982.0       13.0                 0.0   
316417      ZWE 2022-12-29     260015.0       33.0                 0.0   
316418      ZWE 2022-12-30     260059.0       44.0                 0.0   
316419      ZWE 2022-12-31     260100.0       41.0                 0.0   
316420      ZWE 2023-01-01     260146.0       46.0                 0.0   

        people_vaccinated  people_fully_vaccinated  Week  Year  
353                   0.0                     

In [ ]:
covid_cases_df = covid_cases_df.groupby(['iso_code', 'Year', 'Week']).agg({'total_cases':'max', 'new_cases':'sum', 'total_vaccinations':'max', 'people_vaccinated':'sum', 'people_fully_vaccinated':'sum'})
covid_cases_df = pd.DataFrame(covid_cases_df).reset_index()
covid_cases_df.rename(columns={'iso_code':'CountryCode'},inplace=True)
covid_cases_df

,CountryCode,Year,Week,total_cases,new_cases,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,ABW,2020,1,0.0,0.0,0.0,0.0,0.0
1,ABW,2020,2,0.0,0.0,0.0,0.0,0.0
2,ABW,2020,3,0.0,0.0,0.0,0.0,0.0
3,ABW,2020,4,0.0,0.0,0.0,0.0,0.0
4,ABW,2020,5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
46016,ZWE,2023,20,264936.0,95.0,0.0,0.0,0.0
46017,ZWE,2023,21,265023.0,87.0,0.0,0.0,0.0
46018,ZWE,2023,22,265139.0,116.0,0.0,0.0,0.0
46019,ZWE,2023,23,265139.0,0.0,0.0,0.0,0.0


In [ ]:
covid_cases_df['total_vaccinations'

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
46016    0.0
46017    0.0
46018    0.0
46019    0.0
46020    0.0
Name: total_vaccinations, Length: 46021, dtype: float64

In [ ]:
mortality_df = mortality_df[mortality_df['Sex'] == 'b']
mortality_df = mortality_df.set_index(['CountryCode'])
mortality_df.reset_index()

,CountryCode,Year,Week,Sex,DTotal,RTotal
0,AUS,2015,1,b,2925.0,0.006386
1,AUS,2015,2,b,2772.0,0.006052
2,AUS,2015,3,b,2770.0,0.006048
3,AUS,2015,4,b,2768.0,0.006043
4,AUS,2015,5,b,2680.0,0.005851
...,...,...,...,...,...,...
39449,USA,2023,12,b,59943.0,0.009387
39450,USA,2023,13,b,59125.0,0.009259
39451,USA,2023,14,b,57823.0,0.009055
39452,USA,2023,15,b,58082.0,0.009095


In [ ]:
merged_df = pd.merge(mortality_df, covid_cases_df, on=['CountryCode', 'Year', 'Week'])
merged_df

,CountryCode,Year,Week,Sex,DTotal,RTotal,total_cases,new_cases
0,AUS,2020,1,b,2926.0,0.005947,0.0,0.0
1,AUS,2020,2,b,2902.0,0.005898,0.0,0.0
2,AUS,2020,3,b,2914.0,0.005922,0.0,0.0
3,AUS,2020,4,b,3007.0,0.006111,4.0,4.0
4,AUS,2020,5,b,2910.0,0.005914,12.0,8.0
...,...,...,...,...,...,...,...,...
5324,USA,2023,12,b,59943.0,0.009387,102697566.0,152968.0
5325,USA,2023,13,b,59125.0,0.009259,102873924.0,176358.0
5326,USA,2023,14,b,57823.0,0.009055,102977396.0,103472.0
5327,USA,2023,15,b,58082.0,0.009095,103081453.0,104057.0


In [ ]:
merged_df.groupby(['CountryCode']).max()['total_cases']

CountryCode
AUS     11020674.0
AUT      6076436.0
BEL      4798658.0
BGR      1304320.0
CAN      4492624.0
CHE      4405436.0
CHL      5287288.0
CZE      4630803.0
DNK      3413642.0
ESP     13833956.0
EST       618726.0
FIN      1476335.0
GRC      5965643.0
HRV      1272377.0
HUN      2201355.0
ISL       209191.0
ISR      4823337.0
ITA     25680148.0
KOR     19776050.0
LTU      1318474.0
LUX       319959.0
LVA       977827.0
NLD      8610372.0
NOR      1484923.0
POL      6516781.0
PRT      5581621.0
RUS      3159297.0
SVK      1866312.0
SVN      1342692.0
SWE      2704949.0
TWN            0.0
USA    103179920.0
Name: total_cases, dtype: float64